# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [3]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [4]:
import os
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime
import mlrun

> 2020-10-07 09:46:38,742 [warning] Failed resolving version info. Ignoring and using defaults


In [5]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [6]:
# nuclio: end-code

# Test models locally and deploy

The sklearn-project generated one or more models that will be deployed in the server project `sklearn-servers`

### test locally

In [7]:
import cloudpickle as cp
models_path = '/User/ml/demos/sklearn-pipe/pipe/9cf9bf0b-a293-48d9-80be-0e66faa2f9f6/model/1/'

from sklearn.datasets import load_iris
from mlrun.serving.server import get_mock_server
server = get_mock_server()

In [ ]:
server.add_model("mymodel", model_class=ClassifierModel, model_path=models_path)

In [9]:
iris = load_iris()
x = iris['data'].tolist()

In [10]:
result = server.test("mymodel/infer", {"inputs": x})
result.keys()

> 2020-10-07 09:46:49,269 [debug] router run model mymodel, op=infer


dict_keys(['id', 'model_name', 'outputs'])

## document and save

In [11]:
from mlrun import code_to_function
fn = code_to_function('v2-model-server', description="generic sklearn model server",
                     categories=['serving', 'ml'],
                     labels={'author': 'yaronh', 'framework': 'sklearn'},
                     code_output='.')
fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2020-10-07 09:46:56,879 [info] function spec saved to path: function.yaml


##  Deploy server

In [12]:
from mlrun import mount_v3io
fn.apply(mount_v3io())
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True
address = fn.deploy(project='v2-srv')

> 2020-10-07 09:47:20,997 [info] deploy started
[nuclio] 2020-10-07 09:47:28,142 (info) Build complete
[nuclio] 2020-10-07 09:47:34,213 (info) Function deploy complete
[nuclio] 2020-10-07 09:47:34,220 done updating v2-srv-v2-model-server, function address: 3.128.234.166:31483


##  Test server

In [13]:
predict_url = address+"/v2/models/mymodel/infer"
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
!curl {predict_url} -d '{my_data}'

{"id": "6f8c9783-6b01-4f50-80f3-d348c5575edd", "model_name": "mymodel", "outputs": [0, 2]}